## MemEx SRAM Tutorial
How to use the SRAM class in the MemEx package: Import memex (of course) and spidev packages. Spidev makes it easy to talk directly to SPI devices like our serial SRAMs. To initialize a device, you need to specify the SPI port like this:
```
dut = memex.sram(spidev.SpiDev())
```
If you're using more than one device on the SPI bus, it makes sense to declare the SPI bus first before initializing the SRAMs:
```
spi = spidev.SpiDev()
dut0 = memex.sram(spi,cs = 0)
dut1 = memex.sram(spi,cs = 1)
```
With multiple devices on the SPI bus, you'll need to differentiate the chip select (or CS) pins, hence the "cs = x" inputs. You can also use different SPI busses:
```
spi = spidev.SpiDev()
dut0 = memex.sram(spi)
dut1 = memex.sram(spi,cs=1)
dut2 = memex.sram(spi,bus=1)
```
The above lines would initialize 3 SRAMs, on spi0.0, spi0.1, and spi1.0 (these are the 3 available SPI ports on the Raspberry Pi GPIO).

The SRAM class in the MemEx package has some useful functions: <br>
`data = read(address,n=1)` : read n bytes starting from address <br>
`write(address,data,n=1)` : write n copies of data starting at address (data can be a list) <br>
`fill(pattern)` : fill entire memory (based on sram.size) <br>
`faults = check(pattern)` : compare entire memory to given pattern, return count of mismatched bits <br>
`save(file = "data/sram/save.csv")` : save state of entire memory as a single-row csv with given file path <br>

In case you didn't know, function inputs that have a `=` are optional, and if nothing is assigned the function will default the value after the `=`. Functions that don't have a return listed above return 0 on success, and all functions return -1 if an error is detected.

One last thing:

You're welcome to talk directly to the devices with SPI. Here are the four basic steps to do so:
```
sram.spi.open(sram.bus,sram.cs) # open spi comms
sram.spi.max_speed_hz = sram.spi_hz # apply SPI speed limit
miso = sram.spi.xfer(mosi) # transmit MOSI data and receive MISO data
sram.spi.close() # close SPI comms
```
For formatting your MOSI messages, refer to the datasheet: <https://ww1.microchip.com/downloads/en/DeviceDoc/20005142C.pdf>

In [1]:
# MemEx library SRAM object tutorial

import memex
import spidev
import time

# set up comms
spi = spidev.SpiDev()

# init device
sram = memex.sram(spi,bus=0,cs=0,debug=False) # init SRAM on SPI bus 0 and chip select 0 (spi0.0)
# hint: try turning on debug mode

# write and read a single byte
sram.write(0,85) # write 85 (0x55) to address 0
data = sram.read(0) # read from address 0
print(" data:",data)

# write and read a list of data
sram.write(1,[2,4,6,8]) # write a list of data starting at address 4
data = sram.read(1,n=4) # read 4 bytes starting at address 0
print(" data:",data)

# multi-byte write and read with iteration
sram.write(5,170,n=4) # write 170 (0xaa) to 4 bytes starting at address 0
data = sram.read(5,n=4) # read 4 bytes starting at address 0
print(" data:",data)

TypeError: __init__() got an unexpected keyword argument 'bus'

In [2]:
import memex
import spidev
import time

# set up comms
spi = spidev.SpiDev()

# init device
sram = memex.sram(spi,bus=0,cs=0,debug=False) # init SRAM on SPI bus 0 and chip select 0 (spi0.0)

# script setup
data_pattern = 0
rewrite_data = True

# count faults
faults = sram.check(data_pattern) # count mismatched bits
if rewrite_data:
    sram.fill(data_pattern) # fill memory with data pattern
print("faults:",faults)

TypeError: __init__() got an unexpected keyword argument 'bus'

In [17]:
sram.fill(0)

0

In [48]:
import memex
sram = memex.sram(spidev.SpiDev())
sram.save(file = "data/sram/puf.csv")

0

In [46]:
# find read/write speed, read/write current, and idle current

import memex
import spidev
import adafruit_ina260
import busio
import board
import time
import gpiozero
from colorzero import Color

# script setup
pattern = 0
default_hz = 5000000
time_limit = 10

i2c = busio.I2C(board.SCL, board.SDA) # init i2c bus
spi = spidev.SpiDev() # init spi bus
sram = memex.sram(spi,spi_hz = default_hz)
sensor = adafruit_ina260.INA260(i2c, address=0x41) # init power meter (ina260) for sram power
led = gpiozero.RGBLED(red = 22, green = 27, blue = 17, pwm = True)

resolution = 1000000 # starting resolution
led.color = (0.2,0,0.5)
while resolution > 0:
    sram.fill(pattern)
    faults = sram.check(pattern)
    if faults == 0:
        sram.spi_hz += resolution
    elif faults > 0:
        sram.spi_hz -= resolution
        resolution = int(resolution/10)
led.off()
print("max spi:",sram.spi_hz,"Hz")

start = time.time()
read_current = []
read_power = []
kb_read = 0
led.pulse(on_color = Color("red"))
while time.time() - start < time_limit: 
    sram.read(0,n=1024) # read a kilobyte
    kb_read += 1
    read_current += [-sensor.current]
    read_power += [sensor.power]
led.off()
read_speed = kb_read / time_limit # kB/s
read_current = sum(read_current)/len(read_current) # mA
read_power = sum(read_power)/len(read_power) # mW

print("\nread speed:",read_speed,"kB/s")
print("read current:",read_current,"mA")
print("read power:",read_power,"mW")

start = time.time()
write_current = []
write_power = []
kb_written = 0
led.pulse(on_color=Color("green"))
while time.time() - start < time_limit: 
    sram.write(0,pattern,n=1024) # write a kilobyte
    kb_written += 1
    write_current += [-sensor.current]
    write_power += [sensor.power]
led.off()
write_speed = kb_written / time_limit # kB/s
write_current = sum(write_current)/len(write_current) # mA
write_power = sum(write_power)/len(write_power) # mW

print("\nwrite speed:",write_speed,"kB/s")
print("write current:",write_current,"mA")
print("write power:",write_power,"mW")

start = time.time()
idle_current = []
idle_power = []
led.pulse(on_color=Color("blue"))
while time.time() - start < time_limit:
    time.sleep(0.001) # wait 1 ms
    idle_current += [-sensor.current]
    idle_power += [sensor.power]
led.off()
idle_current = sum(idle_current)/len(idle_current) # mA
idle_power = sum(idle_power)/len(idle_power) # mW

print("\nidle current:",idle_current,"mA")
print("idle power:",idle_power,"mW")

sram.spi_hz = default_hz # reset spi max
led.close()

max spi: 19993551 Hz

read speed: 118.9 kB/s
read current: 2.7733389402859547 mA
read power: 8.738435660218672 mW

write speed: 118.2 kB/s
write current: 2.81831641285956 mA
write power: 8.646362098138749 mW

idle current: 2.7963818321785987 mA
idle power: 8.783679753656658 mW


In [28]:
import memex
import time
import spidev
import board
import busio
import digitalio
import adafruit_ina260

spi_disable = digitalio.DigitalInOut(board.D25) # init spi_disable pin
spi_disable.direction = digitalio.Direction.OUTPUT # set to output
spi_disable.value = True # init as false


time.sleep(2)

i2c = busio.I2C(board.SCL, board.SDA) # init i2c bus
sd = adafruit_ina260.INA260(i2c, address=0x41) # init power meter (ina260) for sd power
print("voltage:",sd.voltage)
print("current:",sd.current)
print("power:",sd.power)

time.sleep(1)

spi_disable.value = False

voltage: 1.9937500000000001
current: -3.75
power: 10


In [29]:
import memex
import digitalio
import board
import spidev

spi_disable = digitalio.DigitalInOut(board.D25) # init spi_disable pin
spi_disable.direction = digitalio.Direction.OUTPUT # set to output
spi_disable.value = False # init as false

sram0 = memex.sram(spi=spidev.SpiDev(),debug=False)
sram1 = memex.sram(spi=spidev.SpiDev(),cs=1,debug=False)

sram0.fill(1)
print(sram0.check(0))

sram1.fill(0)
print(sram1.check(255))

32768
262144


In [1]:
import memex
import digitalio
import board
import busio
import time

sram = memex.sram()

start = time.time()
sram.fill(0)
print("faults:",sram.check(1))
print("time:",time.time()-start)

faults: 32768
time: 1.9165842533111572


In [28]:
import memex
import digitalio
import board
import busio
import adafruit_mcp4725
import adafruit_ads1x15
import adafruit_ina260

spi_disable = digitalio.DigitalInOut(board.D25) # init spi_disable pin
spi_disable.direction = digitalio.Direction.OUTPUT # set to output
spi_disable.value = True # initial value

cs0 = digitalio.DigitalInOut(board.D8)
cs0.direction = digitalio.Direction.OUTPUT # set to output
cs0.value = False

cs1 = digitalio.DigitalInOut(board.D7)
cs1.direction = digitalio.Direction.OUTPUT # set to output
cs1.value = False

i2c = busio.I2C(board.SCL, board.SDA)

dac = adafruit_mcp4725.MCP4725(i2c)

adc = adafruit_ads1x15.ads1115.ADS1115(i2c)
a0 = adafruit_ads1x15.analog_in.AnalogIn(adc, adafruit_ads1x15.ads1115.P0)
a1 = adafruit_ads1x15.analog_in.AnalogIn(adc, adafruit_ads1x15.ads1115.P1)
a2 = adafruit_ads1x15.analog_in.AnalogIn(adc, adafruit_ads1x15.ads1115.P2)

sensor = adafruit_ina260.INA260(i2c, address=0x41)

print("pre:",a0.voltage,a1.voltage,a2.voltage,"|",sensor.voltage,sensor.current,sensor.power)
dac.value = 0
time.sleep(1)
print("none:",a0.voltage,a1.voltage,a2.voltage,"|",sensor.voltage,sensor.current,sensor.power)
dac.raw_value = 420
time.sleep(1)
print("half:",a0.voltage,a1.voltage,a2.voltage,"|",sensor.voltage,sensor.current,sensor.power)
dac.raw_value = 4095
time.sleep(1)
print("full:",a0.voltage,a1.voltage,a2.voltage,"|",sensor.voltage,sensor.current,sensor.power)


pre: 4.096 2.779584826197089 2.779584826197089 | 2.7825 12.5 40
none: 0.0026250801110873747 0.006625202185125278 0.006500198370311594 | 0.005 -2.5 0
half: 0.4996402478102969 0.49814020203253273 0.497640186773278 | 0.4975 1.25 0
full: 4.096 2.7995854365672783 2.8002104556413463 | 2.805 12.5 40


In [6]:
# hold at voltage

import memex
import time
import digitalio
import board
import busio
import adafruit_mcp4725
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import adafruit_ina260
import gpiozero

# script setup
pattern = 0
v_hold = float(input("V_HOLD (V): "))
t_hold = input("T_HOLD (s): ")

i2c = busio.I2C(board.SCL, board.SDA)
spi = busio.SPI(board.SCLK,MOSI=board.MOSI,MISO=board.MISO)

spi_disable = digitalio.DigitalInOut(board.D25) # init spi_disable pin
spi_disable.direction = digitalio.Direction.OUTPUT # set to output
spi_disable.value = False # initial value

sram0 = memex.sram(spi,cs=board.D8) # init sram0
sram1 = memex.sram(spi,cs=board.D7) # init sram1

dac = adafruit_mcp4725.MCP4725(i2c) # init dac

adc = ADS.ADS1115(i2c) # init adc
a0 = AnalogIn(adc, ADS.P0)
a1 = AnalogIn(adc, ADS.P1)
a2 = AnalogIn(adc, ADS.P2)

try:
    led = gpiozero.RGBLED(red=17,green=27,blue=22,pwm=True) # init led
except:
    led.close()
    led = gpiozero.RGBLED(red=17,green=27,blue=22,pwm=True) # init led

# fill with pattern
sram0.fill(pattern)
sram1.fill(pattern)

# set voltage
led.color = (0,0,1) # blue
spi_disable.value = True
sram0.cs.value = False
sram1.cs.value = False
v_actual = memex.set_voltage(v_hold)
print("v_actual:",v_actual)

# wait for time
led.color = (0,1,0) # green
if t_hold is "":
    start = time.time()
    input("Press Enter to reset VDD and count faults")
    t_hold = time.time() - start
else:
    time.sleep(float(t_hold))
    
# reset voltage
led.color = (0,0,1) # blue
spi_disable.value = False
sram0.cs.value = True
sram1.cs.value = True
memex.set_voltage(3.3)
time.sleep(1) # let voltage settle

# save data
led.color = (0,0,0) # off
data = [memex.timestamp(),
        t_hold,
        v_actual,
        pattern,
        sram0.check(pattern),
        sram1.check(pattern)]
header = ["AbsTime(s)",
          "HoldTime(s)",
          "HoldVolts(V)",
          "DataPattern",
          sram0.name + "Faults(#)",
          sram1.name + "Faults(#)"]
print(*header,sep='\t')
print(*data,sep='\t')
memex.log(data,header,file="data/sram/check.csv")

V_HOLD (V): 0.31
T_HOLD (s): 1
Error in memex.set_voltage(): timed out
v_actual: 0.31025946836756496
AbsTime(s)	HoldTime(s)	HoldVolts(V)	DataPattern	SRAMFaults(#)	SRAMFaults(#)
1617908711	1	0.31025946836756496	0	1024	0
